In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
import warnings
import os
import DeepFM as dfm
import preprocess as prep
from sklearn.metrics import roc_curve,confusion_matrix,recall_score,roc_auc_score
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import time
warnings.filterwarnings('ignore')
path_model='D:\\kyk-ml\\Recommendation_FactorModel+lightgbm\\DeepFM\\'

In [4]:
delta0=datetime.timedelta(days=3)
delta=datetime.timedelta(days=15)
now=(datetime.datetime.now()-delta0).strftime(format='%Y/%m/%d')
past=(datetime.datetime.now()-delta).strftime(format='%Y/%m/%d')
train,test=prep.load_full_log(past,now,'2021/2/18','2021/2/19',is_test=True)
item_pool=prep.load_item_pool(past,now)

In [10]:
model=dfm.DeepFatorizationMachine(128,10000)
#model.build((None,None))
train_set,test_set=model.data_pipeline(train,item_pool,test_data=test,sampling_ratio=3,user_cols=['user_id','age','gender'],item_cols=['item_id','item_catalog'])
train_set=train_set.shuffle(10000).batch(4096)
test_set=test_set.shuffle(1000).batch(512)

In [14]:
if os.path.exists(path_model+'DeepFM.h5'):
    print('loading model.\n')
    model.predict(test_set)
    model.load_weights(path_model+'DeepFM.h5')
    model.compile(loss=tf.keras.losses.BinaryCrossentropy(),optimizer=tf.keras.optimizers.Adam(0.01),metrics=[dfm.roc_auc,tf.keras.metrics.Recall()])
    model.fit(train_set,epochs=1)
    model.evaluate(test_set)
    model.save_weights(path_model+'DeepFM.h5')
else:
    model.compile(loss=tf.keras.losses.BinaryCrossentropy(),optimizer=tf.keras.optimizers.Adam(0.01),metrics=[dfm.roc_auc,tf.keras.metrics.Recall()])
    model.fit(train_set,epochs=10,validation_data=test_set)
    model.summary()
    model.save_weights(path_model+'DeepFM.h5')

loading model.

68/68 [==============================] - 1s 14ms/step - loss: 0.3464 - roc_auc: 0.5426 - recall_2: 0.0765


In [15]:
user_ids=["'00d481e75bed4c4aa2cdc0799711fe68'","'017a5094fe544967ad557489a3c97189'","'714a4f3891024e1daf6753e01a14cbb8'"]
recmd=model.feeling_lucky(user_ids,topK=10,feature_cols=['user_id','age','gender','weekday','item_id','item_catalog'])

In [16]:
recmd

,,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0
0,00d481e75bed4c4aa2cdc0799711fe68,企速贷,新网好企贷,客金贷,微业贷（仅做展示）,富民银行-富税贷,企速贷,税E贷,客宜贷,渤税贷,小微快贷
1,017a5094fe544967ad557489a3c97189,微业贷（仅做展示）,税E贷,客金贷,小微快贷,新网好企贷,企速贷,平安税金贷（企业主）,富民银行-富税贷,渤税贷,客宜贷
2,714a4f3891024e1daf6753e01a14cbb8,新网好企贷,苏宁银行-微商贷,微业贷（仅做展示）,税E贷,客宜贷,客金贷,企速贷,平安税金贷（企业主）,平安快贷-新一贷,小微快贷
